# Mobility and Storage Time Calculator

The following code takes processed and "cleaned" water masks from a specified working directory and performs a series of operations to calculate the: 1) area-based floodplain reworking timescales (TR) and distribution of channel areas (AW); 2) the sediment storage time distributions (tstor) using the deterministic (TCB) and probabilistic (TFP) approaches; 3) the reach transit times for both tstor approaches; 4) the total sediment transit time (ttot) for both tstor approaches.

Author: James (Huck) Rees; PhD Student, UCSB Geography

Date: April 9, 2025

## Import packages

In [1]:
import os
import numpy as np
import pandas as pd
from natsort import natsorted
import glob as glob_module
import math
import geopandas as gpd
import ast

import re
import fiona
import rasterio
from rasterio.mask import mask
from rasterio import warp
from rasterio.warp import transform_geom, calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling
from pyproj import CRS, Geod

from scipy.stats import linregress
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

import geemap
import ee
from geopy.distance import geodesic
from collections import defaultdict

# Authenticate with Google Earth Engine
ee.Initialize()

## Initialize functions to calculate floodplain reworking timescale (TR), channel overlap decay timescale (TM) and channel area (AW) distributions from mobility sheets

In [32]:
def extract_year(filename):
    pattern = r".*_(\d{4})_DSWE_level_\d+_cleaned.tif"
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    return None

def get_utm_epsg(lon, lat):
    zone_number = int((lon + 180) / 6) + 1
    is_northern = lat >= 0
    return 32600 + zone_number if is_northern else 32700 + zone_number

def get_aw_dist(base_directory, output_directory, reach_range=None):
    os.makedirs(output_directory, exist_ok=True)
    reach_dirs = [d for d in os.listdir(base_directory) if d.startswith("reach_") and os.path.isdir(os.path.join(base_directory, d))]

    for reach_dir in reach_dirs:
        try:
            reach_number = int(reach_dir.split('_')[1])

            if isinstance(reach_range, int) and reach_number != reach_range:
                continue
            elif isinstance(reach_range, tuple) and not (reach_range[0] <= reach_number <= reach_range[1]):
                continue

            cleaned_dir = os.path.join(base_directory, reach_dir, "Cleaned")
            if not os.path.exists(cleaned_dir):
                print(f"Cleaned folder not found for Reach {reach_number}.")
                continue

            tif_files = [f for f in os.listdir(cleaned_dir) if f.endswith(".tif")]
            aw_values = []

            for tif_file in tif_files:
                with rasterio.open(os.path.join(cleaned_dir, tif_file)) as src:
                    data = src.read(1)
                    transform = src.transform
                    bounds = src.bounds
                    centroid_lon = (bounds.left + bounds.right) / 2
                    centroid_lat = (bounds.top + bounds.bottom) / 2
                    utm_epsg = get_utm_epsg(centroid_lon, centroid_lat)

                    dst_crs = CRS.from_epsg(utm_epsg)
                    transform_utm, width, height = calculate_default_transform(
                        src.crs, dst_crs, src.width, src.height, *src.bounds)

                    reprojected = np.empty((height, width), dtype=data.dtype)

                    reproject(
                        source=data,
                        destination=reprojected,
                        src_transform=transform,
                        src_crs=src.crs,
                        dst_transform=transform_utm,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest
                    )

                    pixel_area = abs(transform_utm.a * transform_utm.e)
                    wet_pixel_count = np.sum(reprojected == 1)
                    total_area_m2 = wet_pixel_count * pixel_area
                    aw_values.append(total_area_m2)

            output_df = pd.DataFrame({'a_w': aw_values})
            output_csv = os.path.join(output_directory, f"Reach_{reach_number}_aw_dist.csv")
            output_df.to_csv(output_csv, index=False)

            print(f"Saved corrected a_w totals for Reach {reach_number} to {output_csv}")

        except Exception as e:
            print(f"Error processing reach folder {reach_dir}: {e}")

def load_rasters(directory):
    rasters = {}
    wetted_areas = []
    for filepath in os.listdir(directory):
        if filepath.endswith('.tif'):
            year = extract_year(filepath)
            if year is not None:
                with rasterio.open(os.path.join(directory, filepath)) as src:
                    data = src.read(1)
                    transform = src.transform
                    pixel_area = abs(transform[0] * transform[4])
                    wetted_area_km2 = np.sum(data == 1) * pixel_area / 1e6
                    wetted_areas.append(wetted_area_km2)
                    rasters[year] = (data == 1, pixel_area)
    median_aw = np.median(wetted_areas)
    return dict(sorted(rasters.items())), median_aw

def calculate_reworked_areas(rasters):
    delta_t_areas = defaultdict(list)
    years = sorted(rasters.keys())
    for i in range(len(years)):
        t1 = years[i]
        base_mask, pixel_area = rasters[t1]
        union_mask = np.copy(base_mask)
        for j in range(i + 1, len(years)):
            t2 = years[j]
            current_mask, _ = rasters[t2]
            union_mask = np.logical_or(union_mask, current_mask)
            reworked_pixels = np.sum(union_mask) - np.sum(base_mask)
            delta_t = t2 - t1
            reworked_area_km2 = (reworked_pixels * pixel_area) / 1e6
            delta_t_areas[delta_t].append(reworked_area_km2)
    return delta_t_areas

def calculate_overlap_areas(rasters):
    """
    Calculate the overlapping channel area (AM) between baseline and future channel positions.
    
    For each baseline year, calculates the intersection area with all future years,
    representing pixels that remain channelized between the two time points.
    
    Parameters:
        rasters (dict): Dictionary mapping years to tuples of (binary_mask, pixel_area)
    
    Returns:
        dict: Dictionary mapping delta_t values to lists of overlap areas in km²
    """
    delta_t_areas = defaultdict(list)
    years = sorted(rasters.keys())
    
    for i in range(len(years)):
        t1 = years[i]
        base_mask, pixel_area = rasters[t1]
        
        for j in range(i + 1, len(years)):
            t2 = years[j]
            current_mask, _ = rasters[t2]
            
            # Calculate intersection (overlap) between baseline and current mask
            overlap_mask = np.logical_and(base_mask, current_mask)
            overlap_pixels = np.sum(overlap_mask)
            
            delta_t = t2 - t1
            overlap_area_km2 = (overlap_pixels * pixel_area) / 1e6
            delta_t_areas[delta_t].append(overlap_area_km2)
    
    return delta_t_areas

def greenberg_exponential(x, PR_over_AW, CR):
    return -PR_over_AW * np.exp(-CR * x) + PR_over_AW

def overlap_exponential(x, PM_over_AW, CM):
    """
    Exponential model for overlap decay.
    
    AM/AW = (1 - PM_over_AW) * exp(-CM * x) + PM_over_AW
    
    Args:
        x: Time (years)
        PM_over_AW: Asymptotic minimum of overlap (normalized by active width)
        CM: Overlap decay rate (year^-1)
    
    Returns:
        Predicted overlap area normalized by active width
    """
    return (1 - PM_over_AW) * np.exp(-CM * x) + PM_over_AW

def calculate_pswitch(TM_over_TR,  P10=0.5903, P90=1.7413):
    """
    Map TM:TR to pswitch using 10th-90th percentile range with physical bounds.
    
    Maps P10 (inefficient reworking) → 0.50 (high switching)
    Maps P90 (efficient reworking) → 0.05 (minimal but non-zero switching)
    
    Physical justification for pswitch >= 0.05:
    - All rivers experience some stochasticity (floods, bank failures, cutoffs)
    - Prevents numerical instabilities in Monte Carlo simulations
    - Consistent with observed behavior of highly efficient systems (Yukon ≈ 0.06)
    
    Args:
        TM_over_TR (float): Ratio of overlap decay to reworking timescales
        P10 (float): 10th percentile of TM:TR in dataset (default 0.5903)
        P90 (float): 90th percentile of TM:TR in dataset (default 1.7413)
    
    Returns:
        float: Switching probability [0.05, 0.50]
    """
    # Linear mapping from [P10, P90] to [0.50, 0.05]
    pswitch = 0.50 - 0.45 * ((TM_over_TR - P10) / (P90 - P10))
    
    # Clamp to physical bounds
    pswitch = max(0.05, min(0.50, pswitch))
    
    return pswitch

def calculate_TW(delta_ts, PR_over_AW, CR, subsample_n=20):
    """
    Calculate the linear floodplain reworking timescale (TW) by subsampling 
    the Greenberg exponential fit and performing linear regression.
    
    This timescale represents the time required to rework one channel width
    of floodplain, calculated from the linear approximation of the exponential
    reworking curve.
    
    Parameters:
        delta_ts (list or array): Time intervals from the reworked area data
        PR_over_AW (float): Plateau parameter from Greenberg exponential fit
        CR (float): Decay rate from Greenberg exponential fit (year^-1)
        subsample_n (int): Number of points to subsample for linear regression (default 20)
    
    Returns:
        float: TW, the linear floodplain reworking timescale (years)
    """
    # Define the range for subsampling
    min_dt = min(delta_ts)
    max_dt = max(delta_ts)
    
    # Subsample the fitted exponential curve
    subsample_delta_t = np.linspace(min_dt, max_dt, subsample_n)
    subsample_ar_aw = greenberg_exponential(subsample_delta_t, PR_over_AW, CR)
    
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(subsample_delta_t, subsample_ar_aw)
    
    # Calculate TW as inverse of slope
    TW = 1 / slope
    
    return TW

def plot_mobility_fits(river_name, ds_order, delta_ts_rework, data_rework, PR_over_AW, CR, TR,
                       delta_ts_overlap, data_overlap, PM_over_AW, CM, TM, output_path):
    """
    Create box-and-whisker plots with fitted curves for reworked area and overlap decay.
    
    Parameters:
        river_name (str): Name of the river
        ds_order (int): Reach number
        delta_ts_rework (list): Time intervals for reworked area data
        data_rework (list): List of normalized reworked area distributions for each delta_t
        PR_over_AW (float): Plateau parameter for reworked area fit
        CR (float): Decay rate for reworked area (year^-1)
        TR (float): Floodplain reworking timescale (years)
        delta_ts_overlap (list): Time intervals for overlap area data
        data_overlap (list): List of normalized overlap area distributions for each delta_t
        PM_over_AW (float): Minimum parameter for overlap decay fit
        CM (float): Decay rate for overlap area (year^-1)
        TM (float): Overlap decay timescale (years)
        output_path (str): Path to save the figure
    
    Returns:
        None
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Reworked Area with box plots
    bp1 = ax1.boxplot(data_rework, positions=delta_ts_rework, widths=0.8, patch_artist=True)
    for patch in bp1['boxes']:
        patch.set_facecolor('lightblue')
    
    # Fitted curve for reworked area
    x_fit_rework = np.linspace(min(delta_ts_rework), max(delta_ts_rework), 200)
    y_fit_rework = greenberg_exponential(x_fit_rework, PR_over_AW, CR)
    ax1.plot(x_fit_rework, y_fit_rework, 'r-', linewidth=2, label='Greenberg fit')
    
    # Add equation to plot
    equation_text_1 = f'$A_R/A_W = {PR_over_AW:.3f}(1 - e^{{-{CR:.4f}t}})$\nTR = {TR:.2f} years'
    ax1.text(0.05, 0.95, equation_text_1, transform=ax1.transAxes, 
            fontsize=11, verticalalignment='top', 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax1.set_xlabel('Time interval (years)', fontsize=12)
    ax1.set_ylabel('$A_R / A_W$ (Reworked Area / Active Width)', fontsize=12)
    ax1.set_title(f'{river_name} Reach {ds_order}: Floodplain Reworking', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Overlap Decay with box plots
    bp2 = ax2.boxplot(data_overlap, positions=delta_ts_overlap, widths=0.8, patch_artist=True)
    for patch in bp2['boxes']:
        patch.set_facecolor('lightgreen')
    
    # Fitted curve for overlap decay
    x_fit_overlap = np.linspace(min(delta_ts_overlap), max(delta_ts_overlap), 200)
    y_fit_overlap = overlap_exponential(x_fit_overlap, PM_over_AW, CM)
    ax2.plot(x_fit_overlap, y_fit_overlap, 'b-', linewidth=2, label='Exponential decay fit')
    
    # Add equation to plot
    equation_text_2 = f'$A_M/A_W = {1-PM_over_AW:.3f}e^{{-{CM:.4f}t}} + {PM_over_AW:.3f}$\nTM = {TM:.2f} years'
    ax2.text(0.05, 0.95, equation_text_2, transform=ax2.transAxes, 
            fontsize=11, verticalalignment='top', 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax2.set_xlabel('Time interval (years)', fontsize=12)
    ax2.set_ylabel('$A_M / A_W$ (Overlap Area / Active Width)', fontsize=12)
    ax2.set_title(f'{river_name} Reach {ds_order}: Overlap Decay', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved mobility plots to {output_path}")


def calculate_mobility(river_name, ds_order, working_directory):
    """
    Calculate mobility metrics (TR, TM, CR, CM, TW, pswitch) for a single river reach.
    
    This function analyzes channel migration data to extract floodplain reworking
    and overlap decay characteristics. It fits exponential models to both reworked
    area and overlap area data, then calculates characteristic timescales.
    
    Parameters:
        river_name (str): Name of the river being analyzed.
        ds_order (int): Reach number (downstream order).
        working_directory (str): Base path to the working directory containing RiverMapping data.
    
    Returns:
        dict: Dictionary containing:
              - TR (float): Floodplain reworking timescale (years)
              - TM (float): Overlap decay timescale (years)
              - TW (float): Linear floodplain reworking timescale (years)
              - CR (float): Floodplain reworking decay rate (year^-1)
              - CM (float): Overlap decay rate (year^-1)
    
    Outputs:
        - CSV file: Active width distribution saved to Mobility/{river_name}/AW_distributions/
        - PNG plots: Fitted curves for reworked and overlap areas saved to Mobility/{river_name}/MobilityPlots/
    """
    # Define paths
    base_raster_dir = f"{working_directory}/RiverMapping/RiverMasks/{river_name}"
    reach_dir = f"reach_{ds_order}"
    full_path = os.path.join(base_raster_dir, reach_dir, "Cleaned")
    
    output_dir = f"{working_directory}/RiverMapping/Mobility/{river_name}"
    aw_output_dir = os.path.join(output_dir, "AW_distributions")
    plot_output_dir = os.path.join(output_dir, "MobilityPlots")
    os.makedirs(aw_output_dir, exist_ok=True)
    os.makedirs(plot_output_dir, exist_ok=True)
    
    # Generate AW distribution using get_aw_dist()
    get_aw_dist(base_raster_dir, aw_output_dir, reach_range=ds_order)
    
    # Load the generated AW distribution
    aw_csv_path = os.path.join(aw_output_dir, f"Reach_{ds_order}_aw_dist.csv")
    aw_distribution = pd.read_csv(aw_csv_path)
    
    # Load rasters and calculate median active width
    rasters, median_aw = load_rasters(full_path)
    
    # Calculate reworked areas
    delta_t_areas = calculate_reworked_areas(rasters)
    delta_ts_rework = sorted(delta_t_areas.keys())
    data_rework = [[val / median_aw for val in delta_t_areas[dt]] for dt in delta_ts_rework]
    medians_rework = [np.median(vals) for vals in data_rework]
    
    # Fit Greenberg exponential to reworked area data
    x_data_rework = np.array(delta_ts_rework)
    y_data_rework = np.array(medians_rework)
    initial_guess_rework = [max(y_data_rework), 0.1]
    popt_rework, _ = curve_fit(greenberg_exponential, x_data_rework, y_data_rework, p0=initial_guess_rework)
    PR_over_AW, CR = popt_rework
    
    # Calculate TR
    TR = (1 / CR) * (1 / PR_over_AW)
    
    # Calculate TW (linear floodplain reworking timescale)
    TW = calculate_TW(delta_ts_rework, PR_over_AW, CR)
    
    # Calculate overlap areas
    delta_t_overlap = calculate_overlap_areas(rasters)
    delta_ts_overlap = sorted(delta_t_overlap.keys())
    data_overlap = [[val / median_aw for val in delta_t_overlap[dt]] for dt in delta_ts_overlap]
    medians_overlap = [np.median(vals) for vals in data_overlap]
    
    # Fit overlap exponential to overlap decay data
    x_data_overlap = np.array(delta_ts_overlap)
    y_data_overlap = np.array(medians_overlap)
    initial_guess_overlap = [max(y_data_overlap), 0.1]
    popt_overlap, _ = curve_fit(overlap_exponential, x_data_overlap, y_data_overlap, p0=initial_guess_overlap)
    PM_over_AW, CM = popt_overlap
    
    # Calculate TM
    TM = (1 / CM) * (1 / (1 - PM_over_AW))
    
    # Calculate floodplain reworking efficiency ratio
    TM_over_TR = TM / TR
    
    # Calculate pswitch
    pswitch = calculate_pswitch(TM_over_TR)
    
    # Create plots
    plot_path = os.path.join(plot_output_dir, f"Reach_{ds_order}_mobility_fits.png")
    plot_mobility_fits(river_name, ds_order, delta_ts_rework, data_rework, PR_over_AW, CR, TR,
                       delta_ts_overlap, data_overlap, PM_over_AW, CM, TM, plot_path)
    
    # Return results
    results = {
        'TR': TR,
        'TM': TM,
        'TM_over_TR': TM_over_TR,
        'TW': TW,
        'CR': CR,
        'CM': CM,
        'median_aw': median_aw,
        'PR_over_AW': PR_over_AW,
        'PM_over_AW': PM_over_AW,
        'pswitch': pswitch
    }
    
    return results
    
def get_mobility(csv_path):
    """
    Iterates through a CSV file of river names and paths, calculating mobility metrics
    (TR, TM, TW, CR, CM) for each reach as specified.
    
    Parameters:
        csv_path (str): Path to the CSV file with columns:
                        - river_name
                        - working_directory
                        - reach_range (e.g., "All", "(1, 3)", or "2")
    
    Outputs:
        CSV file: Mobility metrics saved to Mobility/{river_name}/{river_name}_mobility_metrics.csv
                  with columns: ds_order, TR, TM, TW, CR, CM, median_aw, PR_over_AW, PM_over_AW
    """
    river_data = pd.read_csv(csv_path)
    
    for _, row in river_data.iterrows():
        river_name = row['river_name']
        working_directory = row['working_directory']
        reach_range = row['reach_range']
        
        # Define base directory for raster masks
        base_raster_dir = f"{working_directory}/RiverMapping/RiverMasks/{river_name}"
        output_dir = f"{working_directory}/RiverMapping/Mobility/{river_name}"
        os.makedirs(output_dir, exist_ok=True)
        
        # Parse reach_range to determine which reaches to process
        reach_dirs = [d for d in os.listdir(base_raster_dir) if d.startswith("reach_")]
        available_reaches = sorted([int(d.split('_')[1]) for d in reach_dirs])
        
        if isinstance(reach_range, str):
            reach_range = reach_range.strip()
            if reach_range == "All":
                reaches_to_process = available_reaches
            elif reach_range.startswith("(") and reach_range.endswith(")"):
                # Parse tuple format like "(1, 3)"
                reach_start, reach_end = map(int, reach_range.strip("()").split(","))
                reaches_to_process = [r for r in available_reaches if reach_start <= r <= reach_end]
            else:
                # Single reach number
                reaches_to_process = [int(reach_range)]
        elif isinstance(reach_range, int):
            reaches_to_process = [reach_range]
        elif isinstance(reach_range, tuple):
            reach_start, reach_end = reach_range
            reaches_to_process = [r for r in available_reaches if reach_start <= r <= reach_end]
        else:
            raise ValueError(f"Invalid reach_range format: {reach_range}")
        
        # Calculate mobility metrics for each reach
        all_results = []
        
        for ds_order in reaches_to_process:
            try:
                print(f"Processing {river_name} Reach {ds_order}...")
                results = calculate_mobility(river_name, ds_order, working_directory)
                
                # Add ds_order to results
                results['ds_order'] = ds_order
                all_results.append(results)
                
            except Exception as e:
                print(f"Error processing {river_name} Reach {ds_order}: {e}")
                continue
        
        # Compile results into DataFrame
        if all_results:
            results_df = pd.DataFrame(all_results)
            
            # Reorder columns for clarity
            column_order = ['ds_order', 'TR', 'TM', 'TM_over_TR', 'TW', 'CR', 'CM', 'median_aw', 'PR_over_AW', 'PM_over_AW', 'pswitch']
            results_df = results_df[column_order]
            
            # Sort by ds_order
            results_df = results_df.sort_values('ds_order')
            
            # Save to CSV
            output_csv = os.path.join(output_dir, f"{river_name}_mobility_metrics.csv")
            results_df.to_csv(output_csv, index=False)
            print(f"Saved mobility metrics for {river_name} to {output_csv}")
        else:
            print(f"No results generated for {river_name}")

## Initialize functions to calculate first-passage time distributions of sediment storage time (TFP; probablistic tstor estimate)

In [33]:
def import_aw_distribution(river_name, reach_number, working_directory):
    """
    Imports the AW distribution for a specified reach.

    Args:
        river_name (str): Name of the river.
        reach_number (int): Reach number to import AW distribution.
        working_directory (str): Base working directory containing the river data.

    Returns:
        DataFrame: A DataFrame containing the AW distribution for the specified reach.
    """
    # Define base directory for AW distributions
    aw_dir = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, 'AW_Distributions')

    # Ensure the directory exists
    if not os.path.exists(aw_dir):
        raise FileNotFoundError(f"AW distribution directory not found: {aw_dir}")

    # Load AW distribution for the specified reach
    aw_file = os.path.join(aw_dir, f"Reach_{reach_number}_aw_dist.csv")
    if not os.path.isfile(aw_file):
        raise FileNotFoundError(f"AW file not found: {aw_file}")

    aw_distribution = pd.read_csv(aw_file)

    return aw_distribution

def calculate_tstor_distribution(channel_belt_area, aw_distribution, tw, pswitch, num_iterations=10000, max_timesteps=10000):
    """
    Calculates the Tstor distribution for a single reach using the random walk/single event model and returns the result.
    
    The channel undergoes a random walk, sampling AW values from the distribution at each timestep,
    until it returns to its starting position x0.
    
    Args:
        channel_belt_area (float): Channel belt area (in square km) for the reach.
        aw_distribution (DataFrame): DataFrame with column 'a_w' (in square m).
        tw (float): Lateral migration timescale.
        pswitch (float): The probability of the channel switching direction at each timestep tw.
        num_iterations (int): Number of simulations to run (default 10000).
        max_timesteps (int): Maximum timesteps per simulation (default 10000).
    
    Returns:
        DataFrame: A DataFrame containing the TFP distribution.
    """
    # Convert channel belt area from km² to m²
    channel_belt_area_m2 = channel_belt_area * 1_000_000
    
    if aw_distribution.empty:
        raise ValueError("AW distribution is empty.")
    
    # Extract all AW values for random sampling
    aw_values = aw_distribution['a_w'].values
    
    tfp_times = []
    
    for _ in range(num_iterations):
        x0 = np.random.uniform(0, channel_belt_area_m2)
        x = x0
        total_time = 0
        timestep_count = 0
        current_direction = np.random.choice([-1, 1])  # Initialize first direction randomly
        
        while timestep_count < max_timesteps:
            # Sample a new AW value for this timestep
            aw_step = np.random.choice(aw_values)
            
            # Determine direction based on switching probability
            if np.random.random() < pswitch:
                # Switch direction
                current_direction = -current_direction
            # else: keep the same direction
            
            direction = current_direction
            x_intended = x + direction * aw_step
            
            reflection_occurred = False
            
            # Reflect at boundaries
            if x_intended < 0:
                x_new = -x_intended
                reflection_occurred = True
            elif x_intended > channel_belt_area_m2:
                x_new = 2 * channel_belt_area_m2 - x_intended
                reflection_occurred = True
            else:
                x_new = x_intended
            
            # Check if we've crossed x0 (returned to starting position)
            crossed = False
            
            if reflection_occurred:
                if (x_intended < x0 and x_new >= x0) or (x_intended > x0 and x_new <= x0) or (timestep_count > 0 and x == x0 and x_new != x0):
                    crossed = True
            else:
                if (x_new >= x0 and x < x0) or (x_new <= x0 and x > x0):
                    crossed = True
            
            if crossed:
                # Calculate fractional time
                if reflection_occurred:
                    if x_intended < 0:
                        distance_to_boundary = abs(x - 0)
                        distance_from_boundary_to_x0 = abs(x0 - 0)
                        total_distance_to_x0 = distance_to_boundary + distance_from_boundary_to_x0
                    else:
                        distance_to_boundary = abs(channel_belt_area_m2 - x)
                        distance_from_boundary_to_x0 = abs(channel_belt_area_m2 - x0)
                        total_distance_to_x0 = distance_to_boundary + distance_from_boundary_to_x0
                    
                    fractional_tw = (total_distance_to_x0 / aw_step) * tw
                else:
                    remaining_distance = abs(x0 - x)
                    fractional_tw = (remaining_distance / aw_step) * tw
                
                total_time += fractional_tw
                break
            
            total_time += tw
            x = x_new
            timestep_count += 1
        
        if timestep_count < max_timesteps:
            tfp_times.append(total_time)
    
    result_df = pd.DataFrame({'TFP': tfp_times})
    
    return result_df

def get_tstor_distributions(csv_path):
    """
    Processes a range of reaches from a CSV file and calculates TFP distributions for each.

    Args:
        csv_path (str): Path to the CSV file containing river name, working directory, and reach range.

    Outputs:
        CSV files containing TFP distributions for each processed reach.
    """
    # Load the configuration CSV
    config_data = pd.read_csv(csv_path)

    for index, row in config_data.iterrows():
        # Extract river name, reach range, and working directory for each row
        river_name = row['river_name']
        reach_range = row['reach_range']
        working_directory = row['working_directory']
        num_iterations = row['model_iterations']
        max_timesteps = row['max_timesteps']

        # Define directories for required inputs
        channel_belt_file = os.path.join(working_directory, 'ChannelBelts', 'Extracted_ChannelBelts', river_name, f"{river_name}_channelbelt_areas.csv")

        # Check if the required files exist
        if not os.path.isfile(channel_belt_file):
            raise FileNotFoundError(f"Channel belt areas file not found: {channel_belt_file}")
        
        # Load channel belt areas data
        channel_belt_data = pd.read_csv(channel_belt_file)

        # Load mobility metrics file once per river
        mobility_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, f"{river_name}_mobility_metrics.csv")
        if not os.path.isfile(mobility_file):
            raise FileNotFoundError(f"Mobility metrics file not found: {mobility_file}")
        mobility_all_reaches = pd.read_csv(mobility_file)

        # Determine the reach range
        if isinstance(reach_range, str):
            reach_range = reach_range.strip()  # Remove any extra spaces

            if reach_range == "All":
                reach_start = channel_belt_data['ds_order'].min()
                reach_end = channel_belt_data['ds_order'].max()
            elif reach_range.isdigit():
                # Convert a numeric string to an integer
                reach_range = int(reach_range)
                reach_start = reach_range
                reach_end = reach_range
            elif re.match(r'^\(\d{1,4}, \d{1,4}\)$', reach_range):  # Match (XX, YY) with 1 to 4 digits
                try:
                    # Convert the string to a tuple of integers
                    reach_range = ast.literal_eval(reach_range)
                    reach_start, reach_end = reach_range
                except (ValueError, SyntaxError):
                    raise ValueError(f"Invalid reach range format: {reach_range}")
            else:
                raise ValueError(f"Invalid string format for reach_range: {reach_range}")
        elif isinstance(reach_range, (int, float)) and float(reach_range).is_integer():
            # Convert float-like integers (e.g., 7.0) to int
            reach_range = int(reach_range)
            reach_start = reach_range
            reach_end = reach_range
        elif isinstance(reach_range, tuple) and len(reach_range) == 2:
            reach_start, reach_end = reach_range
        else:
            raise ValueError("reach_range must be 'All', an int, or a tuple (start, end).")

        # Generate range of reaches to process
        reaches = range(reach_start, reach_end + 1)

        # Iterate through the range of reaches and calculate TFP for each
        for reach_number in reaches:
            # Get TR and CR values for this reach
            tw = mobility_all_reaches.loc[mobility_all_reaches['ds_order'] == reach_number, 'TW'].values[0]
            pswitch = mobility_all_reaches.loc[mobility_all_reaches['ds_order'] == reach_number, 'pswitch'].values[0]

            # Get channel belt area for the reach
            channel_belt_area = channel_belt_data.loc[channel_belt_data['ds_order'] == reach_number, 'area_sq_km'].values[0]

            # Import AW distribution for the reach
            aw_distribution = import_aw_distribution(river_name, reach_number, working_directory)

            # Calculate the TFP distribution for the reach
            tfp_distribution = calculate_tstor_distribution(channel_belt_area, aw_distribution, tw, pswitch, num_iterations, max_timesteps)

            # Save TFP distribution to a CSV (fixed typo: sinlge -> single)
            output_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, 'Tstor_distributions', f"Reach_{reach_number}_Tstor_distribution.csv")
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            tfp_distribution.to_csv(output_file, index=False)

            print(f"Tstor distribution for Reach {reach_number} saved to {output_file}")

## Initialize functions to run Monte Carlo simulation to calculate total transit time from the number of storage events and storage time distributions

In [34]:
def monte_carlo_reach_transit_time(
    tstor_df,
    transit_df,
    reach_number,
    num_iterations = 10000
    ):
    """
    Monte Carlo simulation of reach transit times using tstor sampling based on fractional 'n_stor'.
    
    Parameters:
        tstor_df (pd.DataFrame): One-column DataFrame of storage time values (e.g. 'Tstor_yr').
        transit_df (pd.DataFrame): DataFrame with 'ds_order' and 'n_stor' columns.
        reach_number (int): Reach number (ds_order) for simulation.
        num_iterations (int): Number of Monte Carlo simulations to run.
    Returns:
        pd.DataFrame: DataFrame of simulated transit times.
    """
    # Clean and check input
    transit_df.columns = transit_df.columns.str.strip()
    if "ds_order" not in transit_df or "n_stor" not in transit_df:
        raise KeyError("transit_df must contain 'ds_order' and 'n_stor' columns.")
    
    if tstor_df.shape[1] != 1:
        raise ValueError("tstor_df must contain exactly one column.")
    
    tstor_vals = tstor_df.iloc[:, 0].dropna().values
    if len(tstor_vals) == 0:
        raise ValueError("No valid storage time data found.")
    
    # Extract n_stor
    n_array = transit_df.loc[transit_df["ds_order"] == reach_number, "n_stor"].values
    if len(n_array) == 0:
        raise ValueError(f"Reach {reach_number} not found in transit_df.")
    
    n = float(n_array[0])
    int_part = int(np.floor(n))
    frac_part = n - int_part
    
    results = []
    for _ in range(num_iterations):
        # Handle the integer part
        if int_part > 0:
            samples = np.random.choice(tstor_vals, size=int_part, replace=True)
            total = samples.sum()
        else:
            total = 0.0
        
        # Handle the fractional part
        if np.random.rand() < frac_part:
            extra = np.random.choice(tstor_vals)
            total += extra
        
        results.append(total)
    
    return pd.DataFrame({"reach_transit_time_yr": results})

def get_reach_transittimes(work_dir: str, river_name: str):
    """
    Processes all reach transit time distributions for a given river.
    
    Parameters:
        work_dir (str): Path to the working directory containing relevant data files.
        river_name (str): Name of the river to process.
    """
    # Path to transit length (storage) values
    nstor_path = os.path.join(work_dir, "RiverMapping", "Mobility", river_name, f"{river_name}_transit_lengths.csv")
    nstor_vals = pd.read_csv(nstor_path)
    
    # Path where Tstor distribution files are stored
    tstor_dir = os.path.join(work_dir, "RiverMapping", "Mobility", river_name, "Tstor_distributions")
    tstor_files = {
        file: pd.read_csv(os.path.join(tstor_dir, file))
        for file in os.listdir(tstor_dir)
        if file.endswith(".csv") and file.startswith("Reach_")
    }
    
    # Prepare output directory
    output_dir = os.path.join(work_dir, "RiverMapping", "Mobility", river_name, "RTT_Distributions")
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each reach
    for filename, tstor_df in tstor_files.items():
        try:
            reach_number = int(filename.split("_")[1])
            reach_transit_time_df = monte_carlo_reach_transit_time(tstor_df, nstor_vals, reach_number)
            output_path = os.path.join(output_dir, f"Reach_{reach_number}_RTT_distribution.csv")
            reach_transit_time_df.to_csv(output_path, index=False)
            print(f"Saved: {output_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

## Calculate distributions for total alluvial transit time

In [35]:
def calculate_ttt_statistics(directory: str):
    """
    Calculates and saves statistics for all total transit time distribution CSV files
    found in the given directory.

    Parameters:
        directory (str): Directory containing TTT distribution CSV files.
    """
    files = [f for f in os.listdir(directory) if f.endswith("_distribution.csv")]

    for file in files:
        file_path = os.path.join(directory, file)
        ttt_df = pd.read_csv(file_path)

        if "total_transit_time_yr" not in ttt_df.columns:
            print(f"Skipping {file} — missing 'total_transit_time_yr' column.")
            continue

        # Compute statistics for all columns
        stats_list = []
        for column in ttt_df.columns:
            stats_list.append({
                "Variable": column,
                "Mean": np.mean(ttt_df[column]),
                "Standard Deviation": np.std(ttt_df[column]),
                "Min": np.min(ttt_df[column]),
                "1st Quartile": np.percentile(ttt_df[column], 25),
                "Median": np.median(ttt_df[column]),
                "3rd Quartile": np.percentile(ttt_df[column], 75),
                "Max": np.max(ttt_df[column])
            })

        stats_df = pd.DataFrame(stats_list)

        # Build output file name
        base_name = os.path.splitext(file)[0]
        stats_file = f"{base_name}_stats.csv"
        stats_path = os.path.join(directory, stats_file)
        stats_df.to_csv(stats_path, index=False)
        print(f"Saved stats: {stats_path}")

def get_total_transit_times(working_dir: str, river_name: str, num_iterations: int = 10_000, reach_start: int = 1, reach_end: int = None):
    """
    Runs a Monte Carlo simulation to compute the total river transit time distribution,
    and includes the sampled reach-level transit times for each iteration.
    
    Parameters:
        working_dir (str): Root directory containing the data folder structure.
        river_name (str): Name of the river for output file naming.
        num_iterations (int): Number of iterations for the Monte Carlo simulation (default is 10,000).
        reach_start (int): Index of the first reach to include (1-based, inclusive).
        reach_end (int): Index of the last reach to include (1-based, inclusive). If None, includes all reaches to the end.
    
    Returns:
        pd.DataFrame: DataFrame containing the total river transit time and individual reach samples.
    """
    rtt_dir = os.path.join(working_dir, 'RiverMapping', 'Mobility', river_name, 'RTT_Distributions')
    all_rtt_files = os.listdir(rtt_dir)
    
    # Determine reach range
    reach_end = reach_end if reach_end is not None else 100
    
    selected_reach_dfs = []
    actual_reaches = []
    
    for reach_num in range(reach_start, reach_end + 1):
        expected_filename = f"Reach_{reach_num}_RTT_distribution.csv"
        file_path = os.path.join(rtt_dir, expected_filename)
        
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if "reach_transit_time_yr" not in df.columns:
                raise KeyError(f"Missing 'reach_transit_time_yr' column in file: {expected_filename}")
            selected_reach_dfs.append(df)
            actual_reaches.append(reach_num)
        else:
            raise FileNotFoundError(f"Expected file not found: {expected_filename}")
    
    simulation_results = []
    
    for _ in range(num_iterations):
        sampled_reach_times = [np.random.choice(df["reach_transit_time_yr"], 1)[0] for df in selected_reach_dfs]
        total_time = sum(sampled_reach_times)
        simulation_results.append(sampled_reach_times + [total_time])
    
    # Build DataFrame with individual reach samples and total time
    columns = [f"reach_{reach}_tt" for reach in actual_reaches] + ["total_transit_time_yr"]
    simulation_df = pd.DataFrame(simulation_results, columns=columns)
    
    # Create output filename reflecting reach range
    reach_range_str = f"R{reach_start}toR{reach_end}"
    output_filename = f"{river_name}_{reach_range_str}_TTT_distribution.csv"
    output_path = os.path.join(working_dir, 'RiverMapping', 'Mobility', river_name, output_filename)
    
    simulation_df.to_csv(output_path, index=False)
    print(f"Saved: {output_path}")
    
    stats_directory = os.path.join(working_dir, 'RiverMapping', 'Mobility', river_name)
    calculate_ttt_statistics(stats_directory)

In [49]:
csv_path = r"D:\Dissertation\Data\RiverMapping\Bermejo_river_datasheet.csv"
working_directory = r"D:\Dissertation\Data"
river_name = "Bermejo"
iterations = 10000
reach_start = 16
reach_end = 20

In [22]:
get_mobility(csv_path)

Processing Beni Reach 1...
Saved corrected a_w totals for Reach 1 to D:\Dissertation\Data/RiverMapping/Mobility/Beni\AW_distributions\Reach_1_aw_dist.csv
Saved mobility plots to D:\Dissertation\Data/RiverMapping/Mobility/Beni\MobilityPlots\Reach_1_mobility_fits.png
Processing Beni Reach 2...
Saved corrected a_w totals for Reach 2 to D:\Dissertation\Data/RiverMapping/Mobility/Beni\AW_distributions\Reach_2_aw_dist.csv
Saved mobility plots to D:\Dissertation\Data/RiverMapping/Mobility/Beni\MobilityPlots\Reach_2_mobility_fits.png
Processing Beni Reach 3...
Saved corrected a_w totals for Reach 3 to D:\Dissertation\Data/RiverMapping/Mobility/Beni\AW_distributions\Reach_3_aw_dist.csv
Saved mobility plots to D:\Dissertation\Data/RiverMapping/Mobility/Beni\MobilityPlots\Reach_3_mobility_fits.png
Processing Beni Reach 4...
Saved corrected a_w totals for Reach 4 to D:\Dissertation\Data/RiverMapping/Mobility/Beni\AW_distributions\Reach_4_aw_dist.csv
Saved mobility plots to D:\Dissertation\Data/Riv

In [26]:
get_tstor_distributions(csv_path)

Tstor distribution for Reach 1 saved to D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Tstor_distributions\Reach_1_Tstor_distribution.csv
Tstor distribution for Reach 2 saved to D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Tstor_distributions\Reach_2_Tstor_distribution.csv
Tstor distribution for Reach 3 saved to D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Tstor_distributions\Reach_3_Tstor_distribution.csv
Tstor distribution for Reach 4 saved to D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Tstor_distributions\Reach_4_Tstor_distribution.csv
Tstor distribution for Reach 5 saved to D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Tstor_distributions\Reach_5_Tstor_distribution.csv
Tstor distribution for Reach 6 saved to D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Tstor_distributions\Reach_6_Tstor_distribution.csv
Tstor distribution for Reach 7 saved to D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Tstor_distributions\Reach_7_Tstor_distribution.csv
Tstor 

In [39]:
get_reach_transittimes(working_directory, river_name)

Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_1_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_2_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_3_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_4_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_5_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_6_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_7_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_8_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_9_RTT_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Beni\RTT_Distributions\Reach_10_

In [50]:
get_total_transit_times(working_directory, river_name, iterations, reach_start, reach_end)

Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Bermejo_R16toR20_TTT_distribution.csv
Saved stats: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Bermejo_R1toR20_TTT_distribution_stats.csv
Saved stats: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Bermejo_R1toR3_TTT_distribution_stats.csv
Saved stats: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Bermejo_R4toR8_TTT_distribution_stats.csv
Saved stats: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Bermejo_R9toR15_TTT_distribution_stats.csv
Saved stats: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\Bermejo_R16toR20_TTT_distribution_stats.csv
